# Topic modelling

## 0.0 Imports

In [1]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dglover\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

## 1.0 Run

In [14]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['use', 'show', 'however', 'approach', 'well', 'provide',' present', 'include', 'word', 'nlp', 'natural', 'language', 'processing'])

In [15]:
# Import Dataset
df = pd.read_csv('..\\data\\summaries_full.csv', index_col=0)
#print(df.target_names.unique())
df.head()

,Summary
PDF URL,
http://arxiv.org/pdf/1608.04434v1,Hadoop is one of the platform s that can proce...
http://arxiv.org/pdf/2202.07138v2,Integrating AI Planning with Natural Language ...
http://arxiv.org/pdf/1906.11608v2,The tools are machine learning based using nat...
http://arxiv.org/pdf/2006.16212v1,The name Tangkhul also known as Hao or Ihao re...
http://arxiv.org/pdf/1511.07916v1,This is a lecture note for the course DS GA at...


In [16]:
df.reset_index(inplace = True)

In [17]:
# Import Dataset
df2 = pd.read_csv('..\\data\\arxiv_papers_full_v2.csv', index_col=0)

In [18]:
df2.head(2)

,Title,PDF URL,Author,DOI,Published Date,Summary,Journal Ref,Primary Category,Category,Entry ID
0,Natural Language Processing using Hadoop and K...,http://arxiv.org/pdf/1608.04434v1,"[arxiv.Result.Author('Emre Erturk'), arxiv.Res...",NaN,2016-08-15 23:09:21+00:00,"Natural language processing, as a data analyti...",NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1608.04434v1
1,Integrating AI Planning with Natural Language ...,http://arxiv.org/pdf/2202.07138v2,"[arxiv.Result.Author('Kebing Jin'), arxiv.Resu...",NaN,2022-02-15 02:19:09+00:00,Natural language processing (NLP) aims at inve...,NaN,cs.AI,"['cs.AI', 'cs.CL']",http://arxiv.org/abs/2202.07138v2


In [19]:
df2.rename(columns = {'Summary' : 'Abstract'}, inplace=True)

In [20]:
#df2['Filename'] = df2['PDF URL'].map(lambda x: x.split('/')[-1] + '.pdf')

In [21]:
df = df.merge(df2, how = 'left', on = 'PDF URL', indicator = True)

In [22]:
df['_merge'].value_counts()

both          3569
left_only        0
right_only       0
Name: _merge, dtype: int64

In [23]:
df.isna().sum()

PDF URL                0
Summary                1
Title                  0
Author                 0
DOI                 3166
Published Date         0
Abstract               0
Journal Ref         3052
Primary Category       0
Category               0
Entry ID               0
_merge                 0
dtype: int64

In [24]:
df.shape

(3569, 12)

In [27]:
df['Summary'] = df['Summary'].map(lambda x: str(x).lower().replace('natural langauge processing', ''))

In [28]:
# Convert to list
data = df['Summary'].values.tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

In [29]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [30]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [31]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [32]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

C:\Users\dglover\Anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [33]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [34]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [35]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.030*"com" + 0.026*"publish" + 0.026*"network" + 0.016*"image" + '
  '0.012*"visual" + 0.011*"author" + 0.009*"academic" + 0.008*"book" + '
  '0.008*"detect" + 0.007*"computer_science"'),
 (1,
  '0.022*"system" + 0.016*"use" + 0.013*"process" + 0.012*"semantic" + '
  '0.012*"problem" + 0.012*"human" + 0.011*"information" + 0.009*"base" + '
  '0.009*"tool" + 0.008*"question"'),
 (2,
  '0.061*"model" + 0.030*"task" + 0.021*"text" + 0.018*"use" + 0.016*"datum" + '
  '0.015*"base" + 0.013*"propose" + 0.011*"large" + 0.010*"learn" + '
  '0.010*"performance"'),
 (3,
  '0.032*"publish" + 0.026*"study" + 0.022*"paper" + 0.016*"research" + '
  '0.014*"learning" + 0.014*"speech" + 0.012*"language" + 0.011*"author" + '
  '0.011*"word" + 0.011*"work"'),
 (4,
  '0.037*"call" + 0.021*"entity" + 0.014*"detail" + 0.013*"role" + '
  '0.012*"visit_local" + 0.012*"key" + 0.011*"attack" + 0.011*"event" + '
  '0.011*"agent" + 0.011*"click"')]


In [36]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.8593679611346

Coherence Score:  0.3311005023982874


In [37]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\dglover\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.251846 -0.103294       1        1  41.293524
1      0.137550 -0.092980       2        1  24.340319
3      0.069507  0.289055       3        1  22.296792
0     -0.243067  0.048212       4        1   7.894840
4     -0.215835 -0.140993       5        1   4.174525, topic_info=                     Term         Freq        Total Category  logprob  loglift
62                  model  4408.000000  4408.000000  Default  30.0000  30.0000
175               publish  1619.000000  1619.000000  Default  29.0000  29.0000
155                  task  2171.000000  2171.000000  Default  28.0000  28.0000
34                  study  1054.000000  1054.000000  Default  27.0000  27.0000
266                system  1012.000000  1012.000000  Default  26.0000  26.0000
...                   ...          ...          ...      ...      ...      ...
2116                stage    37.437866    38.221507   Topic5  -5.2713   3.1555
399            contribute    37.896436    38.691402   Topic5  -5.2591   3.1554
453   prevention_lifeline    35.736219    36.497886   Topic5  -5.3178   3.1551
2279                robot    35.757796    36.522562   Topic5  -5.3172   3.1550
427                   com    64.853706   480.364753   Topic5  -4.7219   1.1738

[250 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2824      4  0.991373  academic
83        4  0.992155    access
814       1  0.997096   achieve
2283      4  0.987989    action
1985      5  0.990823     agent
...     ...       ...       ...
82        1  0.409914      work
82        2  0.025961      work
82        3  0.561582      work
82        4  0.001366      work
2496      4  0.985028       www

[295 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 4, 1, 5])

In [38]:
import pandas as pd

def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [39]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=texts)

C:\Users\dglover\AppData\Local\Temp\ipykernel_18524\1533496723.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


In [40]:
df_topic_sents_keywords

,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
0,2,0.3947,"model, task, text, use, datum, base, propose, ...","[hadoop, platform, process, large, amount, dat..."
1,2,0.6148,"model, task, text, use, datum, base, propose, ...","[integrating, ai, plan, combination, explicit,..."
2,1,0.3330,"system, use, process, semantic, problem, human...","[tool, machine, learning, base, use, rocesse, ..."
3,3,0.4279,"publish, study, paper, research, learning, spe...","[also, know, refer, ethnic, group, manipur, ba..."
4,0,0.2970,"com, publish, network, image, visual, author, ...","[note, course, introduce, reader, neural_netwo..."
...,...,...,...,...
3564,1,0.3844,"system, use, process, semantic, problem, human...","[automate, method, security, directive, presen..."
3565,2,0.4358,"model, task, text, use, datum, base, propose, ...","[modifie, run, driven, augmentation, datum, fu..."
3566,2,0.5882,"model, task, text, use, datum, base, propose, ...","[large, model, llm, revolutionise, understandi..."
3567,2,0.4400,"model, task, text, use, datum, base, propose, ...","[predictive, power, vary, clinical, note, type..."


In [41]:
df

,PDF URL,Summary,Title,Author,DOI,Published Date,Abstract,Journal Ref,Primary Category,Category,Entry ID,_merge
0,http://arxiv.org/pdf/1608.04434v1,hadoop is one of the platform s that can proce...,Natural Language Processing using Hadoop and K...,"[arxiv.Result.Author('Emre Erturk'), arxiv.Res...",NaN,2016-08-15 23:09:21+00:00,"Natural language processing, as a data analyti...",NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1608.04434v1,both
1,http://arxiv.org/pdf/2202.07138v2,integrating ai planning with natural language ...,Integrating AI Planning with Natural Language ...,"[arxiv.Result.Author('Kebing Jin'), arxiv.Resu...",NaN,2022-02-15 02:19:09+00:00,Natural language processing (NLP) aims at inve...,NaN,cs.AI,"['cs.AI', 'cs.CL']",http://arxiv.org/abs/2202.07138v2,both
2,http://arxiv.org/pdf/1906.11608v2,the tools are machine learning based using nat...,Simple Natural Language Processing Tools for D...,[arxiv.Result.Author('Leon Derczynski')],NaN,2019-06-27 13:15:12+00:00,This technical note describes a set of baselin...,NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1906.11608v2,both
3,http://arxiv.org/pdf/2006.16212v1,the name tangkhul also known as hao or ihao re...,Towards the Study of Morphological Processing ...,"[arxiv.Result.Author('Mirinso Shadang'), arxiv...",NaN,2020-06-29 17:24:09+00:00,There is no or little work on natural language...,In proceeding of Regional International Confer...,cs.CL,['cs.CL'],http://arxiv.org/abs/2006.16212v1,both
4,http://arxiv.org/pdf/1511.07916v1,this is a lecture note for the course ds ga at...,Natural Language Understanding with Distribute...,[arxiv.Result.Author('Kyunghyun Cho')],NaN,2015-11-24 23:23:13+00:00,This is a lecture note for the course DS-GA 30...,NaN,cs.CL,"['cs.CL', 'stat.ML']",http://arxiv.org/abs/1511.07916v1,both
...,...,...,...,...,...,...,...,...,...,...,...,...
3564,http://arxiv.org/pdf/2307.01211v1,an automated method for the ontologicalreprese...,An automated method for the ontological repres...,"[arxiv.Result.Author('Giampaolo Bella'), arxiv...",NaN,2023-06-30 09:04:47+00:00,Large documents written in juridical language ...,NaN,cs.AI,"['cs.AI', 'cs.CL']",http://arxiv.org/abs/2307.01211v1,both
3565,http://arxiv.org/pdf/2307.01488v1,scat modifies ran driven augmentations of the ...,SCAT: Robust Self-supervised Contrastive Learn...,"[arxiv.Result.Author('Junjie Wu'), arxiv.Resul...",NaN,2023-07-04 05:41:31+00:00,Despite their promising performance across var...,NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/2307.01488v1,both
3566,http://arxiv.org/pdf/2307.06090v1,large language models llms have revolutionised...,Can Large Language Models Aid in Annotating Sp...,"[arxiv.Result.Author('Siddique Latif'), arxiv....",NaN,2023-07-12 11:27:40+00:00,Despite recent advancements in speech emotion ...,NaN,cs.SD,"['cs.SD', 'eess.AS']",http://arxiv.org/abs/2307.06090v1,both
3567,http://arxiv.org/pdf/2307.07051v1,predictive power varies with clinical note typ...,Making the Most Out of the Limited Context Len...,"[arxiv.Result.Author('Hongyi Zheng'), arxiv.Re...",NaN,2023-07-13 20:04:05+00:00,Recent advances in large language models have ...,Association for Computational Linguistics - St...,cs.CL,"['cs.CL', 'cs.IR', 'cs.LG']",http://arxiv.org/abs/2307.07051v1,both


In [42]:
# Assuming 'original_df' is your original dataframe and 'lda_output' is the output from LDA
merged_df = df.merge(df_topic_sents_keywords, left_index=True, right_index=True)

In [43]:
merged_df.head()

,PDF URL,Summary,Title,Author,DOI,Published Date,Abstract,Journal Ref,Primary Category,Category,Entry ID,_merge,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
0,http://arxiv.org/pdf/1608.04434v1,hadoop is one of the platform s that can proce...,Natural Language Processing using Hadoop and K...,"[arxiv.Result.Author('Emre Erturk'), arxiv.Res...",NaN,2016-08-15 23:09:21+00:00,"Natural language processing, as a data analyti...",NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1608.04434v1,both,2,0.3947,"model, task, text, use, datum, base, propose, ...","[hadoop, platform, process, large, amount, dat..."
1,http://arxiv.org/pdf/2202.07138v2,integrating ai planning with natural language ...,Integrating AI Planning with Natural Language ...,"[arxiv.Result.Author('Kebing Jin'), arxiv.Resu...",NaN,2022-02-15 02:19:09+00:00,Natural language processing (NLP) aims at inve...,NaN,cs.AI,"['cs.AI', 'cs.CL']",http://arxiv.org/abs/2202.07138v2,both,2,0.6148,"model, task, text, use, datum, base, propose, ...","[integrating, ai, plan, combination, explicit,..."
2,http://arxiv.org/pdf/1906.11608v2,the tools are machine learning based using nat...,Simple Natural Language Processing Tools for D...,[arxiv.Result.Author('Leon Derczynski')],NaN,2019-06-27 13:15:12+00:00,This technical note describes a set of baselin...,NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1906.11608v2,both,1,0.3330,"system, use, process, semantic, problem, human...","[tool, machine, learning, base, use, rocesse, ..."
3,http://arxiv.org/pdf/2006.16212v1,the name tangkhul also known as hao or ihao re...,Towards the Study of Morphological Processing ...,"[arxiv.Result.Author('Mirinso Shadang'), arxiv...",NaN,2020-06-29 17:24:09+00:00,There is no or little work on natural language...,In proceeding of Regional International Confer...,cs.CL,['cs.CL'],http://arxiv.org/abs/2006.16212v1,both,3,0.4279,"publish, study, paper, research, learning, spe...","[also, know, refer, ethnic, group, manipur, ba..."
4,http://arxiv.org/pdf/1511.07916v1,this is a lecture note for the course ds ga at...,Natural Language Understanding with Distribute...,[arxiv.Result.Author('Kyunghyun Cho')],NaN,2015-11-24 23:23:13+00:00,This is a lecture note for the course DS-GA 30...,NaN,cs.CL,"['cs.CL', 'stat.ML']",http://arxiv.org/abs/1511.07916v1,both,0,0.2970,"com, publish, network, image, visual, author, ...","[note, course, introduce, reader, neural_netwo..."


In [44]:
merged_df.to_csv('../data/Dominant_topics_of_summaries_v2.csv')

# Archive

In [ ]:
break

In [ ]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=texts)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

In [ ]:
df_dominant_topic.to_csv('../data/Dominant_topics_of_absracts.csv')

In [ ]:
df_dominant_topic.tail(10)

In [ ]:
df_dominant_topic['Dominant_Topic'].value_counts()

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

In [ ]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics